In [ ]:
from imports import *
from dmc import traceReader
%matplotlib inline

In [ ]:
# Use traces with large trigger shifts -- P*S1 channels have -100 to -400 us shifts
trigShift = 'data/FastDMC/TriggerShift/iZIP5-TrigShift_00241024_0000.root'

In [ ]:
# Full set of HV100mm ER events from DC3
import glob
DC3 = glob.glob("data/CDMSlite_TES/weirdEvents/DC3_HV100mm/*.root")

In [ ]:
def bestI0(file, sensor, event, iChan):
    """Compute 'adaptive I0' for trace, scanning the supposed pre-trigger
       baseline for the range of values with the smallest RMS.  This
       should exclude the region where the real trace starts.  Returns
       computed I0 value, along with index of average and RMS value."""
    reader = traceReader(file)
    trace = traceReader.loadTrace(event, sensor, iChan)

    start = 5                   # Need some bins to compute RMS

    rms = [np.std(trace[:i],ddof=1) for i in range(start,len(trace))]
    eom = [np.std(trace[:i])/sqrt(i) for i in range(start,len(trace))]
    ibest = eom.index(min(eom))+start

    return np.mean(trace[:ibest]), ibest, eom[ibest]

In [ ]:
reader = traceReader(DC3)
traces = reader.rawTES(event=-1)
bestI0, err, ibest = reader.bestI0(traces)

In [ ]:
print("Average from 0 to bin",np.mean(ibest),"=", np.mean(bestI0),"+-",np.mean(err))

In [ ]:
event = 7
chans = reader.channels("TES")
for i in range(len(chans)):
    I0, ibase, err = reader.bestI0(reader.rawTES(event, i))
    print(chans[i],": I0 =",I0,"+-",err," to bin",ibase)